In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
df = pd.read_csv('titled_players_raw.csv',index_col=0)

In [3]:
df.head()

,IDcode,Name,T,WT,Oth. T.,Fed,Rtg,Rpd,Blz,B-Year,S,F,Title
0,5813298,"Lim, Kok Ann",CM,,,SGP,,,,1920,M,,"['Candidate Master (CM)', '2012']"
1,4611870,"Keller-Hermann, Edith",WGM,WGM,,GER,2290,,,1921,F,i,"['Woman Grandmaster (WGM)', '1978', 'Woman Int..."
2,2002310,"Levy, Louis",FM,,,USA,2275,,,1921,M,i,[]
3,14100657,"Ousatchi, Mark",FM,,,GER,2157,,,1921,M,i,[]
4,4101529,"Averbakh, Yuri L",GM,,,RUS,2445,,,1922,M,i,"['Grandmaster (GM)', '1952', 'International Ma..."


In [4]:
# Informations à propos des colonnes
for i in df.columns[:12]:
    print(f"{i} : {df[i].unique()}")
    print("Nombre de valeurs unique:", len(df[i].unique()))
    print('\n')

IDcode : [ 5813298  4611870  2002310 ... 44599790   169013 13730150]
Nombre de valeurs unique: 19510


Name : [' Lim, Kok Ann' ' Keller-Hermann, Edith' ' Levy, Louis' ...
 ' Erdogmus, Yagiz Kaan' ' Schnaider, Ilan' ' Zhumagali, Raian']
Nombre de valeurs unique: 19486


T : [' CM' ' WGM' ' FM' ' GM' ' IM' ' WIM' ' WH' ' WFM' ' WCM']
Nombre de valeurs unique: 9


WT : [' ' ' WGM' ' WIM' ' WH' ' WFM' ' WCM']
Nombre de valeurs unique: 6


Oth. T. : [' ' ' FST' ' FT' ' FI' ' DI' ' NI' ' ,FST' ' IO']
Nombre de valeurs unique: 8


Fed : ['SGP' 'GER' 'USA' 'RUS' 'AUT' 'TUN' 'ARG' 'CZE' 'POL' 'NED' 'SRB' 'VEN'
 'SUI' 'BIH' 'ENG' 'EST' 'SLO' 'ISR' 'CRO' 'LAT' 'NCA' 'MEX' 'BUL' 'ITA'
 'BEL' 'MNE' 'HUN' 'FIN' 'UKR' 'IND' 'ROU' 'ESP' 'DEN' 'ISL' 'MDA' 'TKM'
 'CAN' 'FRA' 'NOR' 'PHI' 'AUS' 'ECU' 'SYR' 'LIE' 'ISV' 'MGL' 'BRA' 'GRE'
 'KGZ' 'ARM' 'SVK' 'SWE' 'GEO' 'MKD' 'TTO' 'BLR' 'UZB' 'COL' 'JPN' 'PER'
 'BER' 'IRI' 'ALG' 'THA' 'CUB' 'BAN' 'HKG' 'CHI' 'LTU' 'AFG' 'SCO' 'POR'
 'MNC' 'CHN' 'EGY' 'WLS' '

In [5]:
df[df['T'] == ' WH']

,IDcode,Name,T,WT,Oth. T.,Fed,Rtg,Rpd,Blz,B-Year,S,F,Title
19,1001710,"Vreeken, Corry",WH,WH,,NED,2080,,,1928,F,i,[]


Une seule personne à le titre de WH : woman honorable grandmaster , nous allons supprimer la ligne 

In [6]:
df = df.drop(19)

Comme on peut le voir il y'a plusieurs personnes portant le même nom/prénom mais ce sont bien des personnes distinctes

In [7]:
df[df[['Name']].duplicated(keep=False)].sort_values(by='Name').head(8)

,IDcode,Name,T,WT,Oth. T.,Fed,Rtg,Rpd,Blz,B-Year,S,F,Title
3495,4128869,"Berezin, Viktor",FM,,,RUS,1890,1816,1790,1961,M,,"['Title', 'FIDE PB / Congress', 'Status', 'Int..."
3152,14107872,"Berezin, Viktor",FM,,,UKR,2307,2365,2310,1960,M,i,[]
882,900737,"Djukic, Zeljko",IM,,,SRB,2277,2356,2210,1948,M,,[]
2892,14505185,"Djukic, Zeljko",FM,,,CRO,2263,,2290,1959,M,i,"['FIDE Master (FM)', '2005']"
131,4601530,"Fischer, Johann",FM,,,GER,2063,,,1935,M,i,[]
3212,1600451,"Fischer, Johann",FM,,,AUT,2216,,,1960,M,i,[]
8343,305251,"Gross, David",IM,,,CZE,2373,,,1975,M,,"['International Master (IM)', '1995']"
9310,4628390,"Gross, David",IM,,,GER,2453,2433,,1978,M,i,"['International Master (IM)', '1999']"


In [8]:
df = df.reset_index(drop=True)

In [9]:
df = df.replace({' ':np.nan})

In [10]:
df.head()

,IDcode,Name,T,WT,Oth. T.,Fed,Rtg,Rpd,Blz,B-Year,S,F,Title
0,5813298,"Lim, Kok Ann",CM,NaN,NaN,SGP,NaN,NaN,NaN,1920,M,NaN,"['Candidate Master (CM)', '2012']"
1,4611870,"Keller-Hermann, Edith",WGM,WGM,NaN,GER,2290,NaN,NaN,1921,F,i,"['Woman Grandmaster (WGM)', '1978', 'Woman Int..."
2,2002310,"Levy, Louis",FM,NaN,NaN,USA,2275,NaN,NaN,1921,M,i,[]
3,14100657,"Ousatchi, Mark",FM,NaN,NaN,GER,2157,NaN,NaN,1921,M,i,[]
4,4101529,"Averbakh, Yuri L",GM,NaN,NaN,RUS,2445,NaN,NaN,1922,M,i,"['Grandmaster (GM)', '1952', 'International Ma..."


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19509 entries, 0 to 19508
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   IDcode   19509 non-null  int64 
 1   Name     19509 non-null  object
 2   T        19509 non-null  object
 3   WT       3928 non-null   object
 4   Oth. T.  105 non-null    object
 5   Fed      19509 non-null  object
 6   Rtg      19435 non-null  object
 7   Rpd      11811 non-null  object
 8   Blz      12074 non-null  object
 9   B-Year   19507 non-null  object
 10  S        19509 non-null  object
 11  F        8259 non-null   object
 12  Title    19509 non-null  object
dtypes: int64(1), object(12)
memory usage: 1.9+ MB


La colonne Oth. T. est vide à 99% et nous donne des informations sur les joueurs ayant reçu des titres pour l'arbritrage. Cela ne nous intéresse pas.

Eliminons la

In [12]:
df = df.drop('Oth. T.', axis=1)

La colonne WT est vide à 79%

In [13]:
df['WT'].value_counts(normalize=True, dropna=False)

NaN     0.798657
 WFM    0.091753
 WIM    0.043877
 WCM    0.041314
 WGM    0.024399
Name: WT, dtype: float64

Néanmoins 79% des titres sont détenus par les hommes

In [14]:
df['S'].value_counts(normalize=True)

M    0.797376
F    0.202624
Name: S, dtype: float64

Enlevons les espaces inutiles dans les colonnes de types object

In [15]:
cols = df.select_dtypes(include=['object']).columns
for col in cols:
    df[col] = df[col].str.strip()

In [16]:
df = df.rename(columns = {'IDcode':'id', 
           'Name':'nom',
           'T':'titre',
           'WT':'woman_titre',
           'Fed':'pays',
           'Rtg':'classique',
           'Rpd':'rapide',
           'Blz':'blitz',
           'B-Year':'naissance',
           'S':'sexe',
           'F':'actif',
           'Title':'titre_dates'
          })

In [17]:
df[['classique', 'rapide', 'blitz', 'naissance']] = df[['classique', 'rapide', 'blitz', 'naissance']].astype('float')

Remplaçons la colonne actif par 1 si le joueur est actif 0 sinon

In [18]:
df['actif'].value_counts()

i    8259
Name: actif, dtype: int64

A vérifier si l'in/activité des joueurs semble correct

In [19]:
df['actif'] = df['actif'].replace({'i':'0',np.nan:'1'})


Il faut réfléchir à comment remplacer les obtentions de titres de chaque joueur 

In [20]:
df.iloc[df['titre'].apply(lambda x:len(x)).sort_values(ascending=False)[:100].index].head()

,id,nom,titre,woman_titre,pays,classique,rapide,blitz,naissance,sexe,actif,titre_dates
9754,2219824,"Redondo Arguelles, Graciela",WFM,WFM,ESP,2031.0,1984.0,NaN,1979.0,F,1,"['Woman FIDE Master (WFM)', '2004', 'Title', '..."
11933,13703340,"Uskova, Tatyana",WFM,WFM,KAZ,2126.0,NaN,NaN,1985.0,F,0,"['Woman FIDE Master (WFM)', '2004']"
11971,8602905,"Zhou, Guijue",WIM,WIM,CHN,2225.0,2178.0,NaN,1985.0,F,1,"['Woman International Master (WIM)', '2013', '..."
11968,14105560,"Zemlickova, Olena",WIM,WIM,CZE,2014.0,2122.0,2102.0,1985.0,F,1,"['Woman International Master (WIM)', '2015', '..."
11965,12801496,"Zaksaite, Salomeja",WIM,WIM,LTU,2202.0,2140.0,2002.0,1985.0,F,0,"['Woman International Master (WIM)', '2003']"


Il semblerait que lorsqu'il y'a le mot 'Title' dans la colonne titre_dates, les informations ne sont pas utiles

In [21]:
df.head()

,id,nom,titre,woman_titre,pays,classique,rapide,blitz,naissance,sexe,actif,titre_dates
0,5813298,"Lim, Kok Ann",CM,NaN,SGP,NaN,NaN,NaN,1920.0,M,1,"['Candidate Master (CM)', '2012']"
1,4611870,"Keller-Hermann, Edith",WGM,WGM,GER,2290.0,NaN,NaN,1921.0,F,0,"['Woman Grandmaster (WGM)', '1978', 'Woman Int..."
2,2002310,"Levy, Louis",FM,NaN,USA,2275.0,NaN,NaN,1921.0,M,0,[]
3,14100657,"Ousatchi, Mark",FM,NaN,GER,2157.0,NaN,NaN,1921.0,M,0,[]
4,4101529,"Averbakh, Yuri L",GM,NaN,RUS,2445.0,NaN,NaN,1922.0,M,0,"['Grandmaster (GM)', '1952', 'International Ma..."


In [22]:
# # Change la colonne titre_dates de str à list
df.loc[df['titre_dates'].dropna().index, 'titre_dates'] = df['titre_dates'].dropna().apply(lambda x: ast.literal_eval(x))

In [23]:
def remove_titre(titre_list):
    """Enlève tout les éléments de la lise si 'Title' est présent dans titre_list
    Args :
        titre_list (list): liste de la colonne df['titre_dates']
    
        Returns:
            list : liste modifié si modifications nécessaires, sinon la liste sans changement
        
    
    """
    if 'Title' in titre_list:
        return titre_list[:titre_list.index('Title')]
    return titre_list

In [24]:
df.loc[df['titre_dates'].dropna().apply(remove_titre).index, 'titre_dates'] = df['titre_dates'].dropna().apply(remove_titre)

In [25]:
df['titre_dates'] = df['titre_dates'].apply(lambda x: np.nan if (len(x)==0) else x)

Ca à l'air d'être bon

In [26]:
new_cols = [
            'Woman Candidate Master (WCM)',
            'Candidate Master (CM)',
            'Woman FIDE Master (WFM)',
            'FIDE Master (FM)',
            'Woman International Master (WIM)',
            'International Master (IM)',
            'Woman Grandmaster (WGM)',
            'Grandmaster (GM)'
           ]
dict_cols = {}
for col in new_cols:
    dict_cols[col] = np.nan
dict_default = dict_cols.copy()

In [27]:
# for titre_dates,idx in zip(df['titre_dates'].dropna(),df['titre_dates'].dropna():
#     titre_reshaped = np.reshape(titre_dates, (-1,2))
# #     print(titre_reshaped)
#     dict_cols = dict_default.copy()
#     for i in titre_reshaped:
#         title = i[0]
#         year = i[1]
#         dict_cols[title] = year


In [28]:
def fill_columns(titre_dates):
    titre_reshaped = np.reshape(titre_dates, (-1,2))
    dict_cols = dict_default.copy()
    for i in titre_reshaped:
        title = i[0]
        year = i[1]
        dict_cols[title] = year
    return dict_cols

In [29]:
dict_tofill = df['titre_dates'].dropna().apply(fill_columns)
for i in dict_tofill.index:
    df.loc[i, dict_tofill[i].keys()] = dict_tofill[i].values()


Vérifions que il n'y a pas d'aberrance au niveau des années d'obtentions de titre

In [30]:
for col in df.columns[-8:]:
    print(col,list(df[col].value_counts(normalize=True).sort_index().index),'\n')

Woman Candidate Master (WCM) ['2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'] 

Candidate Master (CM) ['2001', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'] 

Woman FIDE Master (WFM) ['1985', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'] 

FIDE Master (FM) ['0215', '1983', '1984', '1985', '1986', '1987', '1988', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'] 

Woman International Mast

Une petite erreur s'est glissé, remplaçons l'année
FIDE Master (FM) ['0215',


In [31]:
df[df['FIDE Master (FM)'] == '0215']

,id,nom,titre,woman_titre,pays,classique,rapide,blitz,naissance,sexe,actif,titre_dates,Woman Candidate Master (WCM),Candidate Master (CM),Woman FIDE Master (WFM),FIDE Master (FM),Woman International Master (WIM),International Master (IM),Woman Grandmaster (WGM),Grandmaster (GM)
14608,24111180,"Purygin, Andrei",FM,NaN,RUS,2341.0,2266.0,2328.0,1992.0,M,1,"[FIDE Master (FM), 0215]",NaN,NaN,NaN,0215,NaN,NaN,NaN,NaN


In [32]:
df.iloc[14608, 15]
df.iloc[14608, 15] = '2015'

On remarque que le classement minimum pour un joueur est de zéro ce qui est une anomalie, remplaçons les

In [33]:
df.describe()

,id,classique,rapide,blitz,naissance
count,1.950900e+04,19435.000000,11811.000000,12074.000000,19507.000000
mean,6.751109e+06,2214.758117,2172.690797,2162.642206,1978.058287
std,8.226630e+06,220.446269,249.556657,252.017224,17.443011
min,1.000130e+05,0.000000,1039.000000,1088.000000,1920.000000
25%,1.701185e+06,2123.000000,2053.000000,2037.000000,1965.000000
50%,4.155882e+06,2254.000000,2213.000000,2202.000000,1979.000000
75%,1.010010e+07,2348.000000,2331.500000,2323.000000,1992.000000
max,3.712819e+08,2855.000000,2842.000000,2892.000000,2011.000000


In [34]:
df.loc[df[df['classique'] < 1000].index, 'classique'] = df.loc[df[df['classique'] < 1000].index][['rapide','blitz']].mean(axis=1)

On peut voir qu'il y a 2 personnes qui n'ont pas d'année de naissance, mettons les à jour à l'aide d'une recherche google

In [35]:
df.isnull().sum()

id                                      0
nom                                     0
titre                                   0
woman_titre                         15581
pays                                    0
classique                              74
rapide                               7698
blitz                                7435
naissance                               2
sexe                                    0
actif                                   0
titre_dates                          4116
Woman Candidate Master (WCM)        18458
Candidate Master (CM)               17208
Woman FIDE Master (WFM)             17703
FIDE Master (FM)                    12962
Woman International Master (WIM)    18501
International Master (IM)           14643
Woman Grandmaster (WGM)             19075
Grandmaster (GM)                    17848
dtype: int64

In [36]:
df[df['naissance'].isnull()]

,id,nom,titre,woman_titre,pays,classique,rapide,blitz,naissance,sexe,actif,titre_dates,Woman Candidate Master (WCM),Candidate Master (CM),Woman FIDE Master (WFM),FIDE Master (FM),Woman International Master (WIM),International Master (IM),Woman Grandmaster (WGM),Grandmaster (GM)
1110,3200493,"Dekic, Biljana N",WIM,WIM,AUS,1947.0,NaN,1970.0,NaN,F,1,"[Woman International Master (WIM), 1990]",NaN,NaN,NaN,NaN,1990,NaN,NaN,NaN
7418,4200233,"Mastrokoukos, George",IM,NaN,GRE,2306.0,2242.0,2317.0,NaN,M,0,"[International Master (IM), 1994]",NaN,NaN,NaN,NaN,NaN,1994,NaN,NaN


In [37]:
df.iloc[1110,8] = 1950
df.iloc[7418,8] = 1972

Environ 80% des joueurs titrés ont une information concernant l'année d'obtention de titre, cela reste satisfaisant 

In [38]:
df_english = df.rename(columns = {
           'nom':'name',
           'titre':'title',
           'woman_titre':'woman_title',
           'pays':'country',
           'classique':'classical',
           'rapide':'rapid',
           'naissance':'birth_year',
           'sexe':'sex',
           'actif':'active',
           'titre_dates':'title_dates'
          }).copy()

In [39]:
df_english = df_english.drop('title_dates', axis=1)

In [40]:
df_english.to_csv('chess_titled_players.csv')

In [41]:
df.to_csv('titled_players.csv')

# Nettoyage des données pour le classement des joueurs

In [42]:
df_rating = pd.read_pickle('players_rating_clean.zip')

In [43]:
df_rating[df_rating['classique_parties'] > 100]

,id,date,classique_elo,classique_parties,rapide_elo,rapide_parties,blitz_elo,blitz_parties
95614,4146042,2000-01-01,2506.0,152.0,NaN,NaN,NaN,NaN
113928,4137159,2000-01-01,2475.0,104.0,NaN,NaN,NaN,NaN
1656562,3504379,2009-04-01,2399.0,106.0,NaN,NaN,NaN,NaN
1922072,3507610,2009-04-01,2367.0,110.0,NaN,NaN,NaN,NaN
1922076,3507610,2008-04-01,2333.0,109.0,NaN,NaN,NaN,NaN
2066530,3507971,2008-04-01,2351.0,102.0,NaN,NaN,NaN,NaN


In [44]:
df_rating[df_rating['classique_elo'] < 1000]

,id,date,classique_elo,classique_parties,rapide_elo,rapide_parties,blitz_elo,blitz_parties


In [45]:
df_rating.loc[df_rating[df_rating['classique_elo'] < 1000].index, 'classique_elo']

Series([], Name: classique_elo, dtype: float64)

In [46]:
def fill_classique_rating():
    index = df_rating[df_rating['classique_elo'] < 1000].index
    for idx in index:
        id_ = df_rating.loc[idx,'id']
        df_rating[df_rating['id'] == id_]
        l = list(df_rating[df_rating['id'] == id_]['classique_elo'])
        while(0 in l):
            i = l.index(0)
            l[i] = l[i-1]
        df_rating.loc[df_rating[df_rating['id'] == id_].index,'classique_elo'] = l
    return df_rating

In [47]:
df_rating = fill_classique_rating()

In [48]:
df_rating

,id,date,classique_elo,classique_parties,rapide_elo,rapide_parties,blitz_elo,blitz_parties
0,4611870,2021-09-01,2290.0,0.0,NaN,NaN,NaN,NaN
1,4611870,2021-08-01,2290.0,0.0,NaN,NaN,NaN,NaN
2,4611870,2021-07-01,2290.0,0.0,NaN,NaN,NaN,NaN
3,4611870,2021-06-01,2290.0,0.0,NaN,NaN,NaN,NaN
4,4611870,2021-05-01,2290.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2835824,13730150,2017-10-01,NaN,NaN,NaN,NaN,1296.0,0.0
2835825,13730150,2017-09-01,NaN,NaN,NaN,NaN,1296.0,0.0
2835826,13730150,2017-08-01,NaN,NaN,NaN,NaN,1296.0,1.0
2835827,13730150,2017-07-01,NaN,NaN,NaN,NaN,1312.0,0.0


In [49]:
df_rating.to_pickle('players_rating_clean.zip')